In [1]:
%gui qt

# simple plot of a 2D image 

- using Plot2D

## load data from data/lena.hdf5

In [2]:
# input using .hdf5
import h5py
import numpy
dataPath='data/lena.hdf5'
f=h5py.File(dataPath)
from silx.io.utils import h5ls
h5ls(dataPath)

'<HDF5 dataset "lena": shape (512, 512), type "<i8">\n'

In [3]:
image=numpy.array(f['lena'], dtype='float32')

## plot the image
![simple-image](img/plot_lena.png)

In [4]:
from silx.gui.plot import Plot2D
plotImage=Plot2D()
plotImage.addImage(image, origin=(0, 0), legend='sino')  # Plot the RGB image instead of the previous data
plotImage.show()

# display the pixel intensity distribution

## create the histogramnd 
- using silx.math.histogram.Histogramnd

- http://www.silx.org/doc/silx/dev/modules/math/histogram.html

In [5]:
from silx.math.histogram import Histogramnd
histo, w_histo, edges = Histogramnd(image.flatten(), n_bins=256, histo_range=[0,256])

## plot the histogram
- using silx.gui.plot.Plot1d

In [ ]:
from silx.gui.plot import Plot1D
plotHisto = Plot1D()
plotHisto.addCurve(range(256), histo, legend='intensity')
plotHisto.show()

# create a PlotAction which plot the histogram for the current image

- using silx.gui.plot.PlotActions.PlotAction

- doc@ http://www.silx.org/doc/silx/dev/modules/gui/plot/plotactions_examples.html
    
![simple-image](img/plot_image-pixel_intensity.png)

In [6]:
from silx.gui.plot.PlotActions import PlotAction 
from silx.math.histogram import Histogramnd 
from silx.gui.plot import Plot1D

class ComputeHistogramAction(PlotAction):
    """Computes the intensity distribution on the current image

    :param plot: :class:`.PlotWidget` instance on which to operate
    :param parent: See :class:`QAction`
    """
    def __init__(self, plot, parent=None):
        PlotAction.__init__(self,
                            plot,
                            icon='shape-circle',
                            text='pixels intensity',
                            tooltip='Compute image intensity distribution',
                            triggered=self.computeIntensityDistribution,
                            parent=parent)
        self.plotHistogram=Plot1D()

    def computeIntensityDistribution(self):
        """Get the active image and compute the image 
        intensity distribution"""
        # By inheriting from PlotAction, we get access to attribute 
        # self.plot
        # which is a reference to the PlotWindow
        activeImage = self.plot.getActiveImage()

        if activeImage is not None:
            
            histo, w_histo, edges = Histogramnd(activeImage[0].flatten(), 
                                                n_bins=256, 
                                                histo_range=[0,256])
            
            self.plotHistogram.addCurve(range(256), 
                                        histo, 
                                        legend='pixel intensity')
            self.plotHistogram.show()

## Add this action into the toolBar of the window

In [7]:
plotHisto.clear()
myaction=ComputeHistogramAction(plotImage)
toolBar=plotImage.toolBar()
toolBar.addAction(myaction)
plotImage.show()

NameError: name 'plotHisto' is not defined

# show automatically the histogram when the image change

- using plotImage.sigActiveImageChanged.connect(plotHisto)

In [8]:
from silx.math.histogram import Histogramnd 

def computeIntensityDistribution():
        """Get the active image and compute the image 
        intensity distribution"""
        # By inheriting from PlotAction, we get access to attribute 
        # self.plot
        # which is a reference to the PlotWindow
        activeImage = plotImage.getActiveImage()

        if activeImage is not None:
            histo, w_histo, edges = Histogramnd(activeImage[0].flatten(), 
                                                n_bins=256, 
                                                histo_range=[0,256])
            
            from silx.gui.plot import Plot1D
            plotHistogram = Plot1D()
            plotHistogram.addCurve(range(256), 
                                   histo, 
                                   legend='pixel intensity')
            plotHistogram.show()

In [9]:
plotImage=Plot2D()
plotImage.sigActiveImageChanged.connect(computeIntensityDistribution)
plotImage.addImage(image, origin=(0, 0), legend='lena') 

'lena'

In [ ]:
from silx.gui.plot.PlotActions import PlotAction 
from silx.math import histogram
from silx.gui.plot import Plot1D

class ComputeHistogramAction(PlotAction):
    """Computes the intensity distribution on the current image

    :param plot: :class:`.PlotWidget` instance on which to operate
    :param parent: See :class:`QAction`
    """
    def __init__(self, plot, parent=None):
        PlotAction.__init__(self,
                            plot,
                            icon='shape-circle',
                            text='pixels intensity',
                            tooltip='Compute image intensity distribution',
                            triggered=self.computeIntensityDistribution,
                            checkable=True,
                            parent=parent)
        self.plotHistogram=Plot1D()
        self.plot.sigActiveImageChanged.connect(self.update)
        
    def update(self):
        # By inheriting from PlotAction, we get access to attribute 
        # self.plot
        # which is a reference to the PlotWindow
        activeImage = self.plot.getActiveImage()

        if activeImage is not None:
            
            histo, w_histo, edges = histogram.Histogramnd(activeImage[0].flatten(), 
                                                          n_bins=256, 
                                                          histo_range=[0,256])
            
            self.plotHistogram.addCurve(range(256), 
                                        histo, 
                                        legend='pixel intensity')
            
    def computeIntensityDistribution(self):
        """Get the active image and compute the image intensity distribution"""
        if self.isChecked():
            self.update()
            self.plotHistogram.show()
        else:
            self.plotHistogram.hide()

In [ ]:
plotImage=Plot2D()
myaction=ComputeHistogramAction(plotImage)
toolBar=plotImage.toolBar()
toolBar.addAction(myaction)
plotImage.addImage(image, origin=(0, 0), legend='lena')  
plotImage.show()

In [ ]:
plotImage.addImage(image/2.0, origin=(0, 0), legend='lena')  

# For information : the class diagram of the Plot module

![plot class diagram](img/plotGraph.png)